In [ ]:
!nvidia-smi

In [ ]:
from ultralytics import YOLO
import torch
import torchvision
import gc
import roboflow
from roboflow import Roboflow
import time
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import os



CUDA AND TORCH CHECKING

In [ ]:
print(torch.cuda.get_device_name(0))
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Is CUDA available: {torch.cuda.is_available()}")

tensor = torch.rand(3,3).cuda()
print(tensor)

In [ ]:
# Sample data
boxes = torch.tensor([[0, 0, 1, 1], [0.9, 0.9, 2, 2]], dtype=torch.float32).cuda()
scores = torch.tensor([0.9, 0.8], dtype=torch.float32).cuda()

# Perform NMS
nms_indices = torchvision.ops.nms(boxes, scores, iou_threshold=0.5)
nms_indices = nms_indices.cuda()
print(f"NMS indices: {nms_indices}")

In [ ]:
torch.cuda.is_available()
torch.version.cuda
torch.cuda.get_device_name()

TRAINING

In [ ]:
gc.collect()
torch.cuda.empty_cache()

if __name__=='__main__':
    model = YOLO(r'D:\Projects\Helmet detection\Model\yolov8m-seg.pt')

    model.train(data=r'D:\Projects\Helmet detection\Data\data.yaml', epochs=50, imgsz=480, batch=4)

In [ ]:
model=YOLO(r'')
res=model(source=r'D:\Projects\Helmet detection\Data\test',save=False,conf=0.8)

PREDICT

In [ ]:
rf = Roboflow(api_key="AlBM3nHUZMJlOEAokjuW")
project = rf.workspace().project("helmet-and-nonhelmet-detection")
model = project.version(1).model

# Predict using the model
result = model.predict(r"D:\Projects\Helmet detection\Data\train\images\_tmp_44d613c041772528ae5b2f76238cb3844827view_jpg.rf.c0c647c91e6740db45b8eceaa2a20fb7.jpg", confidence=90, overlap=10,labels=True)
predictions = result[0]['confidence']
print(predictions)

In [1]:
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from roboflow import Roboflow
import os
import time

# Function to generate unique filenames for temporary images
def get_unique_filename(file_path):
    base, extension = os.path.splitext(file_path)
    counter = 1
    new_file_path = file_path
    while os.path.exists(new_file_path):
        new_file_path = f"{base}_{counter}{extension}"
        counter += 1
    return new_file_path

# Pre-load models once outside the loop
rf_motorcycle = Roboflow(api_key="lNRNQAqP9F0FHx3Bokha")
project_motorcycle = rf_motorcycle.workspace("vens-shell").project("motorcycle-detection-ctu3w")
model_motorcycle = project_motorcycle.version(2).model

rf_helmet = Roboflow(api_key="AlBM3nHUZMJlOEAokjuW")
project_helmet = rf_helmet.workspace().project("helmet-and-nonhelmet-detection")
model_helmet = project_helmet.version(1).model

# Open the webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default webcam

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

frame_count = 0
skip_frames = 5  # Detect only every 5th frame to speed up the process

# Main loop for real-time detection
while True:
    start_time = time.time()  # Measure processing time per frame
    ret, frame = cap.read()  # Capture a frame from the webcam
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)

    if frame_count % skip_frames == 0:  # Only detect every 'skip_frames' frames
        # Save the current frame as a temporary image
        temp_image_path = get_unique_filename(r"D:\Projects\Helmet detection\Predict\Temp_frame\temp_frame.jpg")
        pil_image.save(temp_image_path)

        # Motorcycle detection
        result_motorcycle = model_motorcycle.predict(temp_image_path, confidence=50, overlap=10)
        predictions_motorcycle = result_motorcycle.predictions

        for motorcycle in predictions_motorcycle:
            moto_bbox = motorcycle['x'], motorcycle['y'], motorcycle['width'], motorcycle['height']
            moto_x0, moto_y0 = moto_bbox[0] - moto_bbox[2] / 2, moto_bbox[1] - moto_bbox[3] / 2
            moto_x1, moto_y1 = moto_bbox[0] + moto_bbox[2] / 2, moto_bbox[1] + moto_bbox[3] / 2

            # Crop motorcycle region for helmet detection
            motorcycle_image = pil_image.crop((moto_x0, moto_y0, moto_x1, moto_y1))
            temp_motorcycle_path = get_unique_filename(r"D:\Projects\Helmet detection\Predict\Temp_frame\temp_motorcycle.jpg")
            motorcycle_image.save(temp_motorcycle_path)

            # Helmet detection in cropped region
            result_helmet = model_helmet.predict(temp_motorcycle_path, confidence=70, overlap=10, labels=True)
            predictions_helmet = result_helmet.predictions

            # Draw bounding boxes for helmets and non-helmets
            for prediction in predictions_helmet:
                bbox = prediction['x'] + moto_x0, prediction['y'] + moto_y0, prediction['width'], prediction['height']
                x0, y0 = bbox[0] - bbox[2] / 2, bbox[1] - bbox[3] / 2
                x1, y1 = bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2

                label = f"({prediction['confidence']*100:.2f}%)"
                color = "green" if prediction['class'] == "Helmet" else "red"

                # Draw the rectangle and label for helmet/non-helmet
                draw = ImageDraw.Draw(pil_image)
                draw.rectangle([x0, y0, x1, y1], outline=color, width=2)
                draw.rectangle((x0, y0 - 20, x0 + len(label)*14, y0), fill=color)
                draw.text((x0, y0 - 20), label, fill="black", font=ImageFont.load_default())

            os.remove(temp_motorcycle_path)  # Clean up temporary cropped motorcycle image
        os.remove(temp_image_path)  # Clean up temporary frame image

    # Convert back to OpenCV format and display the frame
    frame_bgr = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    display_width = 1280
    aspect_ratio = frame_bgr.shape[1] / frame_bgr.shape[0]
    display_height = int(display_width / aspect_ratio)
    resized_frame = cv2.resize(frame_bgr, (display_width, display_height))
    # cv2.imshow('Helmet Detection', frame_bgr)

    print(f"Frame processed in {time.time() - start_time:.2f} seconds.")  # Measure time per frame

    # Increment frame count
    frame_count += 1

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Frame processed in 6.39 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.02 seconds.
Frame processed in 2.24 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.02 seconds.
Frame processed in 0.06 seconds.
Frame processed in 0.01 seconds.
Frame processed in 4.26 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.04 seconds.
Frame processed in 3.51 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 1.03 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 0.01 seconds.
Frame processed in 2.58 seconds.
Frame processed in 0.01 se

KeyboardInterrupt: 

In [ ]:
start_time = time.time()
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import os

def get_unique_filename(file_path):
    base, extension = os.path.splitext(file_path)
    counter = 1
    new_file_path = file_path
    while os.path.exists(new_file_path):
        new_file_path = f"{base}_{counter}{extension}"
        counter += 1
    return new_file_path

# Kết nối với model motorcycle
rf_motorcycle = Roboflow(api_key="lNRNQAqP9F0FHx3Bokha")
project_motorcycle = rf_motorcycle.workspace("vens-shell").project("motorcycle-detection-ctu3w")
model_motorcycle = project_motorcycle.version(2).model

# Kết nối với model helmet
rf_helmet = Roboflow(api_key="AlBM3nHUZMJlOEAokjuW")
project_helmet = rf_helmet.workspace().project("helmet-and-nonhelmet-detection")
model_helmet = project_helmet.version(1).model

# Dự đoán xe máy sử dụng mô hình
result_motorcycle = model_motorcycle.predict(r"D:\Projects\Helmet detection\Sample\Image\sample3.jpg", confidence=50, overlap=10)
predictions_motorcycle = result_motorcycle.predictions

# Mở hình ảnh
image_path = r"D:\Projects\Helmet detection\Sample\Image\sample3.jpg"
image = Image.open(image_path)
image_width, image_height = image.size
draw = ImageDraw.Draw(image)

# Dynamically set bounding box thickness and font size based on image size
box_thickness = max(1, int(min(image_width, image_height) * 0.005))  # Scale thickness by 0.5% of the smaller dimension
font_size = max(10, int(min(image_width, image_height) * 0.006))  # Scale font size by 3% of the smaller dimension

# Font để hiển thị nhãn (tùy chọn)
try:
    font = ImageFont.truetype("arial.ttf", font_size)
except IOError:
    font = ImageFont.load_default()

# Vẽ các hộp dự đoán lên ảnh
for motorcycle in predictions_motorcycle:
    moto_bbox = motorcycle['x'], motorcycle['y'], motorcycle['width'], motorcycle['height']
    moto_x0, moto_y0 = moto_bbox[0] - moto_bbox[2] / 2, moto_bbox[1] - moto_bbox[3] / 2
    moto_x1, moto_y1 = moto_bbox[0] + moto_bbox[2] / 2, moto_bbox[1] + moto_bbox[3] / 2

    motorcycle_image = image.crop((moto_x0, moto_y0, moto_x1, moto_y1))  # Cắt vùng ưu tiên và lưu tạm thời
    temp_image_path = get_unique_filename(r"D:\Projects\Helmet detection\Predict\Temp_frame\temp_motorcycle.jpg")
    motorcycle_image.save(temp_image_path)

    # Dự đoán helmet trong vùng ưu tiên
    result_helmet = model_helmet.predict(temp_image_path, confidence=70, overlap=10, labels=True)
    predictions_helmet = result_helmet.predictions
    
    # Vẽ bounding box và label cho mũ bảo hiểm
    for prediction in predictions_helmet:
        bbox = prediction['x'] + moto_x0, prediction['y'] + moto_y0, prediction['width'], prediction['height']
        x0, y0 = bbox[0] - bbox[2] / 2, bbox[1] - bbox[3] / 2
        x1, y1 = bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2
        
        label = f"({prediction['confidence']*100:.2f}%)"
        
        # Use textbbox to get the bounding box size of the text
        label_bbox = draw.textbbox((0, 0), label, font=font)
        label_width = label_bbox[2] - label_bbox[0] + 10  # Add padding around text
        label_height = label_bbox[3] - label_bbox[1] + 5  # Label height based on font size with some padding
        
        if prediction['class'] == "Helmet":
            draw.rectangle([x0, y0, x1, y1], outline="green", width=box_thickness)
            draw.rectangle((x0, y0 - label_height, x0 + label_width, y0), fill="green")
            draw.text((x0 + 5, y0 - label_height), label, fill="black", font=font)
        elif prediction['class'] == "Nonhelmet":
            draw.rectangle([x0, y0, x1, y1], outline="red", width=box_thickness)
            draw.rectangle((x0, y0 - label_height, x0 + label_width, y0), fill="red")
            draw.text((x0 + 5, y0 - label_height), label, fill="black", font=font)
    
    os.remove(temp_image_path)

# Hiển thị ảnh đã dự đoán
plt.figure(figsize=(20, 20))  # Increase figure size to make the window larger
plt.imshow(image)
plt.axis('off')  # Tắt trục
plt.show()

# Lưu ảnh đã chú thích nếu cần
path = get_unique_filename(r"D:\Projects\Helmet detection\Predict\Image\predict.jpg")
image.save(path)

print("Execution time: %.2f seconds" % (time.time() - start_time))

In [4]:
def get_unique_filename(file_path):
    base, extension = os.path.splitext(file_path)
    counter = 1
    new_file_path = file_path
    while os.path.exists(new_file_path):
        new_file_path = f"{base}_{counter}{extension}"
        counter += 1
    return new_file_path

# Paths
video_path = r"Sample\Video\market_vn.mp4"
output_path = r"Predict\Video\result.mp4"
temp_image_base = r"D:\Projects\Helmet detection\Predict\Temp_frame\temp_motorcycle.jpg"

# Check whether directory exists
os.makedirs(os.path.dirname(temp_image_base), exist_ok=True)

# Video setup
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Load Roboflow models
rf_motorcycle = Roboflow(api_key="lNRNQAqP9F0FHx3Bokha")
model_motorcycle = rf_motorcycle.workspace("vens-shell").project("motorcycle-detection-ctu3w").version(2).model

rf_helmet = Roboflow(api_key="AlBM3nHUZMJlOEAokjuW")
model_helmet = rf_helmet.workspace().project("helmet-and-nonhelmet-detection").version(1).model

# Main loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    result_motorcycle = model_motorcycle.predict(frame, confidence=50, overlap=30).json()
    predictions_motorcycle = result_motorcycle["predictions"]

    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size

    box_thickness = max(1, int(min(image_width, image_height) * 0.009))
    font_size = max(10, int(min(image_width, image_height) * 0.006))

    all_predictions_helmet = []

    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()

    for motorcycle in predictions_motorcycle:
        x, y, w, h = motorcycle['x'], motorcycle['y'], motorcycle['width'], motorcycle['height']
        moto_x0, moto_y0 = x - w / 2, y - h / 2
        moto_x1, moto_y1 = x + w / 2, y + h / 2

        motorcycle_image = image.crop((moto_x0, moto_y0, moto_x1, moto_y1))
        motorcycle_array = np.array(motorcycle_image)
        result_helmet = model_helmet.predict(motorcycle_array, confidence=60, overlap=30, labels=True)
        all_predictions_helmet.extend(result_helmet.predictions)

        for prediction in predictions_helmet:
            px, py, pw, ph = prediction['x'] + moto_x0, prediction['y'] + moto_y0, prediction['width'], prediction['height']
            x0, y0 = px - pw / 2, py - ph / 2
            x1, y1 = px + pw / 2, py + ph / 2
            label = f"({prediction['confidence'] * 100:.1f}%)"

            label_bbox = draw.textbbox((0, 0), label, font=font)
            label_w = label_bbox[2] - label_bbox[0] + 10
            label_h = label_bbox[3] - label_bbox[1] + 5

            color = "green" if prediction['class'] == "Helmet" else "red"
            draw.rectangle([x0, y0, x1, y1], outline=color, width=box_thickness)
            draw.rectangle([x0, y0 - label_h, x0 + label_w, y0], fill=color)
            draw.text((x0 + 5, y0 - label_h), label, fill="black", font=font)

    # Draw FPS
    elapsed_time = time.time() - start_time
    current_fps = 1.0 / elapsed_time if elapsed_time > 0 else 0
    fps_text = f"{current_fps:.2f} FPS"
    processed_frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Count helmets and non-helmets
    helmet_count = sum(1 for pred in all_predictions_helmet if pred['class'] == "Helmet")
    nonhelmet_count = sum(1 for pred in all_predictions_helmet if pred['class'] == "Nonhelmet")

    # Format for text
    fps_text = f"{current_fps:.2f} FPS"
    timestamp_text = f"Time: {cap.get(cv2.CAP_PROP_POS_MSEC) / 1000:.1f}s"
    helmet_text = f"Helmet: {helmet_count}"
    nonhelmet_text = f"Nonhelmet: {nonhelmet_count}"

    # Print text on screen
    cv2.putText(processed_frame, fps_text, (frame_width - 180, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.putText(processed_frame, timestamp_text, (frame_width - 180, frame_height - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.putText(processed_frame, helmet_text, (10, frame_height - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.putText(processed_frame, nonhelmet_text, (10, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    out.write(processed_frame)

    print(f"Processed frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))} / {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}")

# Release resources
cap.release()
out.release()


loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Processed frame 1 / 294
Processed frame 2 / 294
Processed frame 3 / 294
Processed frame 4 / 294
Processed frame 5 / 294
Processed frame 6 / 294
Processed frame 7 / 294
Processed frame 8 / 294
Processed frame 9 / 294
Processed frame 10 / 294
Processed frame 11 / 294
Processed frame 12 / 294
Processed frame 13 / 294
Processed frame 14 / 294
Processed frame 15 / 294
Processed frame 16 / 294
Processed frame 17 / 294
Processed frame 18 / 294
Processed frame 19 / 294
Processed frame 20 / 294
Processed frame 21 / 294
Processed frame 22 / 294
Processed frame 23 / 294
Processed frame 24 / 294
Processed frame 25 / 294
Processed frame 26 / 294
Processed frame 27 / 294
Processed frame 28 / 294
Processed frame 29 / 294
Processed frame 30 / 294
Processed frame 31 / 294
Processed frame 32 / 294
Processed frame 33 / 294
Processed frame 34 / 294
Processed frame 35 / 294
Processed frame 36

HTTPError: 403 Client Error: Forbidden for url: https://detect.roboflow.com/motorcycle-detection-ctu3w/2?api_key=lNRNQAqP9F0FHx3Bokha&name=YOUR_IMAGE.jpg&overlap=30&confidence=50&stroke=1&labels=false&format=json